## Notebook for making predictions with deleted constant y samples


In [14]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [15]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [16]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [17]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

#catModel_bc = CatBoostRegressor()

#catModel_abc = CatBoostRegressor()

In [18]:
X_BC_train = pd.concat([X_B_train, X_C_train])
Y_BC_train = pd.concat([Y_B_train, Y_C_train])

X_ABC_train = pd.concat([X_A_train, X_BC_train])
Y_ABC_train = pd.concat([Y_A_train, Y_BC_train])

In [19]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

#catModel_bc.fit(X_BC_train,Y_BC_train)

#catModel_abc.fit(X_ABC_train,Y_ABC_train)

Learning rate set to 0.078443
0:	learn: 851.2531023	total: 22.4ms	remaining: 22.4s
1:	learn: 799.8181176	total: 28.6ms	remaining: 14.3s
2:	learn: 752.7211819	total: 36.4ms	remaining: 12.1s
3:	learn: 710.1928089	total: 42.7ms	remaining: 10.6s
4:	learn: 671.2844644	total: 50.5ms	remaining: 10.1s
5:	learn: 636.2809623	total: 57.4ms	remaining: 9.51s
6:	learn: 604.4052614	total: 63.9ms	remaining: 9.07s
7:	learn: 575.5492186	total: 71.4ms	remaining: 8.85s
8:	learn: 550.0767983	total: 77.9ms	remaining: 8.58s
9:	learn: 526.9052572	total: 84.5ms	remaining: 8.37s
10:	learn: 506.4464584	total: 90.9ms	remaining: 8.17s


11:	learn: 487.5901459	total: 98.2ms	remaining: 8.08s
12:	learn: 471.0521568	total: 104ms	remaining: 7.88s
13:	learn: 456.0507175	total: 110ms	remaining: 7.74s
14:	learn: 443.1110732	total: 116ms	remaining: 7.62s
15:	learn: 430.8190234	total: 123ms	remaining: 7.56s
16:	learn: 420.2237097	total: 131ms	remaining: 7.58s
17:	learn: 410.9534068	total: 137ms	remaining: 7.5s
18:	learn: 402.5760983	total: 145ms	remaining: 7.46s
19:	learn: 395.1010692	total: 151ms	remaining: 7.39s
20:	learn: 388.3378233	total: 158ms	remaining: 7.38s
21:	learn: 382.6851223	total: 165ms	remaining: 7.32s
22:	learn: 377.3155904	total: 172ms	remaining: 7.3s
23:	learn: 372.5972477	total: 180ms	remaining: 7.31s
24:	learn: 368.3701300	total: 188ms	remaining: 7.33s
25:	learn: 364.5931578	total: 194ms	remaining: 7.28s
26:	learn: 361.3569537	total: 200ms	remaining: 7.22s
27:	learn: 358.0331759	total: 207ms	remaining: 7.18s
28:	learn: 355.3856146	total: 212ms	remaining: 7.1s
29:	learn: 353.0604198	total: 218ms	remaining: 7

In [20]:
# Feature importance
#catModel_a.get_feature_importance(prettified=True)

## Preds

In [21]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

#pred_B = catModel_bc.predict(X_B_test)
#pred_C = catModel_bc.predict(X_C_test)

# pred_A = catModel_abc.predict(X_A_test)
# pred_B = catModel_abc.predict(X_B_test)
# pred_C = catModel_abc.predict(X_C_test)

C:\Users\loghe\AppData\Local\Temp\ipykernel_9256\3683585043.py:8: RuntimeWarning: invalid value encountered in divide
  pred_A = catModel_abc.predict(X_A_test) * dm.amplitude[0] / dm.amplitude[1]


In [22]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_abc_2.csv")

df.shape

(2160, 1)